In [1]:
%run "../Utils.ipynb"

In [2]:
%run "./Utils.ipynb"

In [3]:
connection = get_connection()[0]

In [4]:
# Identify the most active trading hour for each contract over the past 7 days.
query = """
SELECT 
  contract_address, 
  hour, 
  num_transactions 
FROM 
  (
    SELECT 
      contract_address, 
      hour, 
      num_transactions, 
      RANK() OVER (
        PARTITION BY contract_address 
        ORDER BY 
          num_transactions DESC
      ) AS Rank 
    FROM 
      (
        SELECT 
          contract_address, 
          STRFTIME('%H', timestamp) AS hour, 
          COUNT(transaction_id) AS num_transactions 
        FROM 
          nft_transactions 
        WHERE 
          timestamp > DATE(
            date('now'), 
            '-7 DAY'
          ) 
        GROUP BY 
          contract_address, 
          STRFTIME('%H', timestamp)
      )
  ) 
WHERE 
  Rank = 1;
"""
get_result(query, connection)

,contract_address,hour,num_transactions
0,0x0acf7fc611d9d2ebbf9d437476eee258b0610c33,20,86
1,0x0b658f3dcfe1a2bb4c98262d199cfc63b8908468,20,96
2,0x0c05e51b42a3abfada28db86f7a2ccdb1b849485,20,76
3,0x0cd8960f729fa4fb641744e920fb94f2da5fdf58,20,73
4,0x0e152238d1176e592d7f2e2a857cb9342780e0dd,20,70
...,...,...,...
95,0xf42c746a3c0ecb820fcd06b0cdbe0f916a5e2fa3,20,79
96,0xf557ebf055a9b5863a8fd95dbf804cb7ddd7687d,20,55
97,0xf5a614e7ddcc44e90b7b68675bead22fc103e5ec,20,73
98,0xfbc5b845dbaeaf22d1cf3e60e673362378b31a3d,20,77


In [5]:
# Compute liquidity for each contract as `total trade value / unique NFTs traded
query = """
SELECT
    contract_address,
    SUM(price) / count(DISTINCT token_id) AS liquidity
FROM nft_transactions
GROUP BY contract_address
ORDER BY liquidity DESC
LIMIT 3
"""

get_result(query, connection)

,contract_address,liquidity
0,0x2e38cdd04ef456f727c861ea8d6043e94558da85,18.401919
1,0xf557ebf055a9b5863a8fd95dbf804cb7ddd7687d,18.072465
2,0x541b966d6703f1406d20198906884e95fcca2c98,17.998726


In [6]:
# For NFTs traded more than once, calculate the average trade price and standard deviation of prices.  
# Display NFTs with a price standard deviation exceeding 30% of the average price.

query = """
SELECT
   token_id,
   AVG(price) AS avg_traded_price,
   SQRT(AVG(price * price) - AVG(price) * AVG(price)) AS "standard_deviation"
FROM nft_transactions
GROUP BY token_id
HAVING 
    COUNT(1) > 1 AND
    standard_deviation > (0.3 * avg_traded_price)
"""

get_result(query, connection)

,token_id,avg_traded_price,standard_deviation
0,0017cfa1-f83b-4fc0-b7dc-e1de7e993c5b,4.328333,1.348992
1,00488c9d-0e1b-42ad-a9b9-7bc10645b0bd,5.620523,2.530255
2,006fa7d6-6bca-49fa-8b9b-c5d7986a5b75,5.923000,2.594575
3,00e3f2c9-ca3d-4c4c-afe8-3fa907c10566,11.972727,19.795545
4,01035a70-6b8b-45c7-a9e7-83653cf0a676,5.854000,2.001905
...,...,...,...
970,ff4be7d4-0911-4bc8-bb45-db7e6e6e5067,18.687143,30.135194
971,ff5de0cc-6cc8-4fab-9fd8-6ac8d60a37bc,4.931818,2.224622
972,ff7ad0f8-979f-4354-9ad1-bee73ea7f949,5.405931,2.640309
973,ffb608a9-85b4-4cf9-abb3-7d0b365bdfff,12.871667,19.496126


In [7]:
# Identify the top 5 sellers with the highest profits from NFTs sold across chains.  
# Include seller address, total profit, number of chains, and chain with the highest profit.

query = """

WITH profits AS (
    SELECT 
        t2.seller_address,
        t2.chain_id,
        SUM(t2.price - t1.price) AS "profit_from_chain_highest",
        count(t2.chain_id) OVER (PARTITION BY t2.seller_address) AS num_chains,
        SUM(t2.price - t1.price) OVER (PARTITION BY t2.seller_address) AS profit,
        RANK() OVER (PARTITION BY t2.seller_address ORDER BY SUM(t2.price - t1.price) DESC) AS RANK
    FROM nft_transactions t1
    JOIN nft_transactions t2
    ON 
        t1.buyer_address = t2.seller_address AND
        t1.token_id = t2.token_id AND
        t1.chain_id <> t2.chain_id AND
        t1.timestamp < t2.timestamp
    GROUP BY t2.seller_address, t2.chain_id
)

--SELECT * FROM profits ORDER BY profit desc

SELECT 
    seller_address,
    profit as total_profit,
    num_chains,
    chain_id chain_highest_profit,
    profit_from_chain_highest
FROM profits
WHERE RANK = 1
GROUP BY seller_address
ORDER BY total_profit DESC
LIMIT 5







"""

get_result(query, connection)

,seller_address,total_profit,num_chains,chain_highest_profit,profit_from_chain_highest
0,0x4288c7753342713efeeab721f7df6a2487d57f15,105.19,93,603da69c-fa30-41ac-b63e-71c2741c2c4b,42.20
1,0xdcaf488187411a8bbf100553d9053a0018e4ea5a,98.32,91,0b1aee59-f097-48f5-8d54-53c08a3f1941,52.20
2,0xc91a32ddaf5a592c49052118208d6d47a0d01bb5,83.97,90,9697e940-0c65-45b2-bd29-da7d391510e2,53.37
3,0x268fe876503303fdcda1645262419c6a07146403,80.07,90,8928a8b1-869e-42e2-9f35-5f049ca8a692,48.15
4,0x8b1d1ce48b5d071b875ec57e8462f57b2b3bac23,76.96,87,a328f70a-a10a-4222-a25b-e10c3af93c7d,53.15


In [8]:
# Detect transactions with prices >3 times the median price in the last 24 hours.

query = """

WITH calc AS (
    SELECT TRANSACTION_id, price, ROW_NUMBER() OVER (ORDER BY price) as row_num, COUNT(*) OVER () as total_count
    FROM nft_transactions
)
SELECT
	transaction_id,
	price,
    CASE
        WHEN total_count % 2 = 0 THEN
            (
                (SELECT price FROM calc WHERE row_num = total_count / 2) +
                (SELECT price FROM calc WHERE row_num = (total_count / 2) + 1)
            ) / 2.0
        ELSE
            (SELECT price FROM calc WHERE row_num = (total_count + 1) / 2)
    END AS median
FROM calc
WHERE price > (3 * median)
ORDER BY price desc
LIMIT 3;

"""

get_result(query, connection)

,TRANSACTION_id,price,median
0,4ac73cc5-65a1-41ae-aa3d-5507fc98ba5e,99.90,5.55
1,163cb5e8-0c72-435e-9333-4eaf85afe7ac,99.86,5.55
2,7edd2bac-9f63-433e-a91b-95d8b01067ea,99.65,5.55


In [9]:
connection.close()